In [1]:
import os

from bids import BIDSLayout

from glob import glob

from nipype.interfaces.io import BIDSDataGrabber
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import Function

In [2]:
bids_dir = os.path.join('/home/tourbier/Desktop/VboxShare','ds-supermri','bids')
output_dir = os.path.join('/home/tourbier/Desktop/VboxShare','ds-supermri','bids','derivatives','mialsrtk')

subject = 'eyesVD46a'

layout = BIDSLayout(bids_dir)
print(layout)

BIDS Layout: ...top/VboxShare/ds-supermri/bids | Subjects: 1 | Sessions: 0 | Runs: 6


In [3]:
bg = Node(interface=BIDSDataGrabber(infields = ['subject']),name='bids_grabber')
bg.inputs.base_dir = bids_dir
bg.inputs.subject = subject
bg.inputs.output_query = {'T2ws': dict(suffix='T2w')}

In [4]:
def filterNiftis(paths):
    paths = [p for p in paths if 'nii' in p]
    print("\n\nLoaded T2w images " + str(paths) + "\n\n")
    return paths

filterT2wNii = Node(Function(function=filterNiftis, input_names=["paths"],
                            output_names=["new_paths"]), name="filterT2wNii")

wf = Workflow(name="bids_demo",base_dir=output_dir)
wf.connect(bg, "T2ws", filterT2wNii, "paths")
res = wf.run()
wf.write_graph()

190322-18:04:13,898 nipype.workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging', 'monitoring']
190322-18:04:13,914 nipype.workflow INFO:
	 Running serially.
190322-18:04:13,914 nipype.workflow INFO:
	 [Node] Setting-up "bids_demo.bids_grabber" in "/home/tourbier/Desktop/VboxShare/ds-supermri/bids/derivatives/mialsrtk/bids_demo/bids_grabber".
190322-18:04:13,933 nipype.workflow INFO:
	 [Node] Running "bids_grabber" ("nipype.interfaces.io.BIDSDataGrabber")
190322-18:04:14,19 nipype.workflow INFO:
	 [Node] Finished "bids_demo.bids_grabber".
190322-18:04:14,20 nipype.workflow INFO:
	 [Node] Setting-up "bids_demo.filterT2wNii" in "/home/tourbier/Desktop/VboxShare/ds-supermri/bids/derivatives/mialsrtk/bids_demo/filterT2wNii".
190322-18:04:14,52 nipype.workflow INFO:
	 [Node] Outdated cache found for "bids_demo.filterT2wNii".
190322-18:04:14,67 nipype.workflow INFO:
	 [Node] Running "filterT2wNii" ("nipype.interfaces.utility.wrappers.Function")


Loaded T2w images ['

'/home/tourbier/Desktop/VboxShare/ds-supermri/bids/derivatives/mialsrtk/bids_demo/graph.png'

In [5]:
from traits.api import *
from nipype.utils.filemanip import split_filename

from nipype.interfaces.base import traits, isdefined, CommandLine, CommandLineInputSpec,\
    TraitedSpec, File, InputMultiPath, OutputMultiPath, BaseInterface, BaseInterfaceInputSpec

class prepareDockerPathsInputSpec(BaseInterfaceInputSpec):
    local_paths = InputMultiPath(File(desc='input paths', mandatory = True, exists = True, arg_str="-i %s"))
    local_dir = Directory(mandatory=True)
    docker_dir = Directory('/fetaldata',mandatory=True)

class prepareDockerPathsOutputSpec(TraitedSpec):
    docker_paths = OutputMultiPath(File(desc='docker paths'))

class prepareDockerPaths(BaseInterface):
    input_spec = prepareDockerPathsInputSpec
    output_spec = prepareDockerPathsOutputSpec

    def _run_interface(self,runtime):
        return runtime
           
    def _list_outputs(self):

        outputs = self._outputs().get()
        outputs["docker_paths"] = []
        for p in self.inputs.local_paths:
            p = os.path.join(self.inputs.docker_dir,p.split(self.inputs.local_dir)[1].strip("/"))
            print(p)
            outputs["docker_paths"].append(p)
        return outputs
    

preparePaths = Node(interface=prepareDockerPaths(), name="preparePaths")  

preparePaths.inputs.local_dir = bids_dir
preparePaths.inputs.docker_dir = '/fetaldata'

wf.connect(filterT2wNii, "new_paths", preparePaths, "local_paths")
res = wf.run()

190322-18:04:14,226 nipype.workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging', 'monitoring']
190322-18:04:14,245 nipype.workflow INFO:
	 Running serially.
190322-18:04:14,246 nipype.workflow INFO:
	 [Node] Setting-up "bids_demo.bids_grabber" in "/home/tourbier/Desktop/VboxShare/ds-supermri/bids/derivatives/mialsrtk/bids_demo/bids_grabber".
190322-18:04:14,264 nipype.workflow INFO:
	 [Node] Running "bids_grabber" ("nipype.interfaces.io.BIDSDataGrabber")
190322-18:04:14,343 nipype.workflow INFO:
	 [Node] Finished "bids_demo.bids_grabber".
190322-18:04:14,344 nipype.workflow INFO:
	 [Node] Setting-up "bids_demo.filterT2wNii" in "/home/tourbier/Desktop/VboxShare/ds-supermri/bids/derivatives/mialsrtk/bids_demo/filterT2wNii".
190322-18:04:14,373 nipype.workflow INFO:
	 [Node] Outdated cache found for "bids_demo.filterT2wNii".
190322-18:04:14,389 nipype.workflow INFO:
	 [Node] Running "filterT2wNii" ("nipype.interfaces.utility.wrappers.Function")


Loaded T2w image

In [6]:
import subprocess
def run(self, command, env={}, cwd=os.getcwd()):
    merged_env = os.environ
    merged_env.update(env)
    process = subprocess.Popen(command, stdout=subprocess.PIPE,
                               stderr=subprocess.STDOUT, shell=True,
                               env=merged_env, cwd=cwd)
    while True:
        line = process.stdout.readline()
        line = str(line)[:-1]
        print(line)
        if line == '' and process.poll() != None:
            break
    if process.returncode != 0:
        raise Exception("Non zero return code: %d"%process.returncode)

In [7]:
from traits.api import *
from nipype.utils.filemanip import split_filename

from nipype.interfaces.base import traits, isdefined, CommandLine, CommandLineInputSpec,\
    TraitedSpec, File, InputMultiPath, OutputMultiPath, BaseInterface, BaseInterfaceInputSpec

import nibabel as nib

class BtkNLMDenoisingInputSpec(BaseInterfaceInputSpec):
    bids_dir = Directory(desc='BIDS root directory',mandatory=True,exists=True)
    in_file = File(desc='Input image',mandatory=True,)
    out_postfix = traits.Str("_nlm", usedefault=True)
    weight = traits.Float(0.1,desc='NLM weight (0.1 by default)')

class BtkNLMDenoisingOutputSpec(TraitedSpec):
    out_file = File(desc='Denoised image')

class BtkNLMDenoising(BaseInterface):

    input_spec = BtkNLMDenoisingInputSpec
    output_spec = BtkNLMDenoisingOutputSpec
    
    def _run_interface(self, runtime):
        _, name, ext = split_filename(os.path.abspath(self.inputs.in_file))
        out_file = os.path.join('/fetaldata/derivatives/mialsrtk/bids_demo/nlmDenoise', ''.join((name, self.inputs.out_postfix, ext)))
        print(os.getcwd())
        cmd = 'docker run --rm -u {}:{} -v {}:/fetaldata -it sebastientourbier/mialsuperresolutiontoolkit:opengeneva btkNLMDenoising -i {} -o {} -b {}'.format(os.getuid(),os.getgid(),self.inputs.bids_dir,self.inputs.in_file,out_file,self.inputs.weight)
        try:
            print('... cmd: {}'.format(cmd))
            run( cmd, env={})
        except:
            print("    MIALSRTK ERROR: Failed to run btkNLMDenoising")

    def _list_outputs(self):
        outputs = self._outputs().get()
        _, name, ext = split_filename(os.path.abspath(self.inputs.in_file))
        outputs['out_file'] = os.path.join(os.getcwd(), ''.join((name, self.inputs.out_postfix, ext)))
        return outputs

class MultipleBtkNLMDenoisingInputSpec(BaseInterfaceInputSpec):
    bids_dir = Directory(desc='BIDS root directory',mandatory=True,exists=True)
    input_images = InputMultiPath(File(desc='files to be denoised', mandatory = True))
    weight = traits.Float(0.1)
    out_postfix = traits.Str("_nlm", usedefault=True)
    
class MultipleBtkNLMDenoisingOutputSpec(TraitedSpec):
    output_images = OutputMultiPath(File())

class MultipleBtkNLMDenoising(BaseInterface):
    input_spec = MultipleBtkNLMDenoisingInputSpec
    output_spec = MultipleBtkNLMDenoisingOutputSpec

    def _run_interface(self, runtime):
        for input_image in self.inputs.input_images:
            ax = BtkNLMDenoising(bids_dir = self.inputs.bids_dir, in_file = input_image, out_postfix=self.inputs.out_postfix, weight = self.inputs.weight)
            ax.run()
        return runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['output_images'] = glob.glob(os.path.abspath("*.nii.gz"))
        return outputs
    

nlmDenoise = Node(interface=MultipleBtkNLMDenoising(),base_dir=os.path.join(output_dir,'bids_demo'),name='nlmDenoise')
nlmDenoise.inputs.bids_dir = bids_dir
nlmDenoise.inputs.weight = 0.1
                  
wf.connect(preparePaths, "docker_paths", nlmDenoise, "input_images")
res = wf.run()

res.outputs
                  
                  

190322-18:04:14,641 nipype.workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging', 'monitoring']
190322-18:04:14,660 nipype.workflow INFO:
	 Running serially.
190322-18:04:14,661 nipype.workflow INFO:
	 [Node] Setting-up "bids_demo.bids_grabber" in "/home/tourbier/Desktop/VboxShare/ds-supermri/bids/derivatives/mialsrtk/bids_demo/bids_grabber".
190322-18:04:14,681 nipype.workflow INFO:
	 [Node] Running "bids_grabber" ("nipype.interfaces.io.BIDSDataGrabber")
190322-18:04:14,754 nipype.workflow INFO:
	 [Node] Finished "bids_demo.bids_grabber".
190322-18:04:14,754 nipype.workflow INFO:
	 [Node] Setting-up "bids_demo.filterT2wNii" in "/home/tourbier/Desktop/VboxShare/ds-supermri/bids/derivatives/mialsrtk/bids_demo/filterT2wNii".
190322-18:04:14,794 nipype.workflow INFO:
	 [Node] Cached "bids_demo.filterT2wNii" - collecting precomputed outputs
190322-18:04:14,794 nipype.workflow INFO:
	 [Node] "bids_demo.filterT2wNii" found cached.
190322-18:04:14,795 nipype.workflow 

RuntimeError: Workflow did not execute cleanly. Check log for details